In [1]:
from DbManager import DbManager
import pandas as pd

In [28]:
db = DbManager("root", "bandtec123", "localhost", "processamento_db")
data_frame = db.read("SELECT * FROM eventos").astype({'categoria':'category'})
data_frame['categoria'] = data_frame['categoria'].cat.codes
data_frame_categoria = data_frame.groupby(['idConsumidor','categoria'])['idEvento'].count().reset_index()
data_frame_categoria.sort_values('idEvento')

Conectado ao banco


,idConsumidor,categoria,idEvento
0,0,0,1
615,696,5,1
616,697,2,1
617,698,0,1
618,698,5,1
...,...,...,...
200,216,4,2
161,176,1,2
121,133,4,2
577,633,4,3


In [26]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   idEvento      1000 non-null   int64  
 1   idConsumidor  1000 non-null   int64  
 2   idade         1000 non-null   int64  
 3   preco         1000 non-null   float64
 4   nome          1000 non-null   object 
 5   categoria     1000 non-null   int8   
 6   fkCupom       1000 non-null   int64  
 7   statusEvento  1000 non-null   int64  
 8   fkEcommerce   1000 non-null   int64  
 9   dataCompra    1000 non-null   int64  
dtypes: float64(1), int64(7), int8(1), object(1)
memory usage: 71.4+ KB
